In [82]:
import torch
import math
import random
import numpy as np

In [89]:
# autograd globally off
torch.set_grad_enabled(False)

In [84]:
class Module (object):
    def forward (self , * input ):
        raise NotImplementedError
    def backward (self , * gradwrtoutput ):
        raise NotImplementedError
    def param (self):
        return []

In [85]:
def generate_data(num):
#   using pytorch tensor operation
#   return the generated random points and labels
    random_points = torch.rand((num, 2))
    # check if the point is in the circle centered at (0.5,0.5) of radius 1/sqrt(2*math.pi)
    return random_points,torch.Tensor([int(pow(x[0]-0.5,2)+pow(x[1]-0.5,2)<1/(2*math.pi)) for x in random_points])

In [86]:
# def generate_data(num):
# #   return the generated random points and labels
#     random_points=np.random.rand(num,2)
#     # check if the point is in the circle centered at (0.5,0.5) of radius 1/sqrt(2*math.pi)
#     return random_points,[int(pow(x[0]-0.5,2)+pow(x[1]-0.5,2)<1/(2*math.pi)) for x in random_points]

In [87]:
train_num, train_labels=generate_data(1000)
test_num, test_labels=generate_data(1000)

In [88]:
print(train_num.shape)
print(len(train_labels))
print(test_num.shape)
print(len(test_labels))

torch.Size([1000, 2])
1000
torch.Size([1000, 2])
1000
